# SOS, SAIL, SPLASH, and billy precipitation data for 2021-2023 winters.

We need to gather the following data

| Campaign | Location      | Name    | Shield Type                                                                         | Gauge Manufacturer     | Gauge Model                                               |
|----------|---------------|---------|------------------------------------------------------------------------------------|------------------------|-----------------------------------------------------------|
| SAIL     | Gothic        | aosmet  | --                                                                                 | Vaisala                | WXT520/530                                                |
| SAIL     | Gothic        | ld      | --                                                                                 | OTT Hydromet Group     | Parsivel2                                                 |
| SAIL     | Gothic        | met     | --                                                                                 | Optical Scientific     | ORG-815-DS Optical Precipitation Gauge                    |
| SAIL     | Gothic        | met     | Single Alter                                                                       | Novalynx               | 260-2500E-12 Tipping Bucket Rain Gauge                    |
| SAIL     | Gothic        | met     | --                                                                                 | Vaisala                | PWD22                                                     |
| SAIL     | Gothic        | wbpluvio| Double fence? Unknown. Recommended for double fence, but possibly single alter     | OTT Hydromet Group     | Pluvio2-L weighing bucket rain gauge                      |
| SPLASH   | Kettle Ponds  | ld      | --                                                                                 | OTT Hydromet Group     | Parsivel2                                                 |
| SPLASH   | Kettle Ponds  | pluvio  | SDFIR                                                                              | OTT Hydromet Group     | Pluvio2-L weighing bucket rain gauge                      |
| SPLASH   | Kettle Ponds-A| ASFS    | --                                                                                 | Unknown                | Unknown                                                   |
| SPLASH   | Avery Picnic  | ASFS    | --                                                                                 | Unknown                | Unknown                                                   |
| billy    | Gothic        | pluvio  | Single Alter                                                                       | Unknown                | Unknown                                                   |
```

In [ ]:
# general
import os
import glob
import datetime as dt
import json
# data 
import xarray as xr 
from sublimpy import utils, variables, tidy, turbulence
import numpy as np
import pandas as pd
from act import discovery, plotting
# plotting
import matplotlib.pyplot as plt
from metpy.cbook import get_test_data
from metpy.plots import add_metpy_logo, SkewT
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.io as pio
# helper tools
from scripts.get_sail_data import get_sail_data
from scripts.helper_funcs import create_windrose_df, simple_sounding, mean_sounding
import scripts.helper_funcs as hf
from metpy import calc, units
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
import time
# make plotly work 
init_notebook_mode(connected=True)
cf.go_offline()

# SAIL

### Setup to download SAIL data

In [ ]:
# Function to load ARM credentials
def load_arm_credentials(credential_path):
    with open(credential_path, 'r') as f:
        credentials = json.load(f)
    return credentials
# Location of ARM credentials
credential_path = '/home/dlhogan/.act_config.json'
credentials = load_arm_credentials(credential_path)
# api token and username for ARM
api_username = credentials.get('username')
api_token = credentials.get('token')

sail_datastream_dict = {
    # "aosmet":"gucaosmetM1.a1", # Too short for analysis
    "met":"gucmetM1.b1",
    "laser_disdrometer_gothic":"gucldM1.b1",
    "ldquants":"gucldquantsM1.c1",
    "pluvio2":"gucwbpluvio2M1.a1",
}

In [ ]:
winter_22 = ('20211001','20220930')
winter_23 = ('20221001','20230930')

In [ ]:
# Set the location of the data folder where this data will be stored
winter_22_folder = 'winter_21_22'
winter_23_folder = 'winter_22_23'
 # change to location of data folder on your machine
storage_directory = f'/storage/dlhogan/synoptic_sublimation/'
# create a sail_data folder if it does not exist
if not os.path.exists(os.path.join(storage_directory,'sail_data')):
    os.makedirs(os.path.join(storage_directory,'sail_data'))
# create a folder for the event if it does not exist
if not os.path.exists(os.path.join(storage_directory,'sail_data',winter_22_folder)):
    os.makedirs(os.path.join(storage_directory,'sail_data',winter_22_folder))
if not os.path.exists(os.path.join(storage_directory,'sail_data',winter_22_folder,'radiosonde')):
    os.makedirs(os.path.join(storage_directory,'sail_data',winter_22_folder,'radiosonde'))
# create a folder for the event if it does not exist
if not os.path.exists(os.path.join(storage_directory,'sail_data',winter_23_folder)):
    os.makedirs(os.path.join(storage_directory,'sail_data',winter_23_folder))
    # make a radiosonde folder if it does not exist
if not os.path.exists(os.path.join(storage_directory,'sail_data',winter_23_folder,'radiosonde')):
    os.makedirs(os.path.join(storage_directory,'sail_data',winter_23_folder,'radiosonde'))

### Download winter 2022 data from SAIL
For now we will just get met, ecor, and laser disdrometer data

In [ ]:
# load in the winter 22 data
sail_winter_22_folder = os.path.join(storage_directory,'sail_data',winter_22_folder)
# create empty data dictionary
w22_data_loc_dict = {}
# Iterate through the dictionary and pull the data for each datastream
for k,v in sail_datastream_dict.items():
    if (k =='radiosonde') & (len(os.listdir(os.path.join(sail_winter_22_folder,"radiosonde"))) > 0):
        print("Radiosonde data already donwloaded.")
        print('-------------------')
    # Check if the file already exists
    elif (os.path.exists(f'{sail_winter_22_folder}/{k}_{winter_22[0]}_{winter_22[1]}.nc')): 
        print(f'{k}_{winter_22[0]}_{winter_22[1]}.nc already exists')
        print('-------------------')
        # add the filename to the dictionary which can be used if we want to load the data
        w22_data_loc_dict[k] = os.path.join(sail_winter_22_folder,f'{k}_{winter_22[0]}_{winter_22[1]}.nc')
        continue
    else:
        # explicitly download radiosonde data because they are a lot easier to process and think about when in individual files
        if k == 'radiosonde':
            discovery.download_data(
                api_username,
                api_token,
                v,
                startdate=winter_22[0],
                enddate=winter_22[1],
                output=sail_winter_22_folder+'/radiosonde/'
            )
        else:
            ds = get_sail_data(api_username,
                        api_token,
                        v,
                        startdate=winter_22[0],
                        enddate=winter_22[1])
            ds.to_netcdf(f'{sail_winter_22_folder}/{k}_{winter_22[0]}_{winter_22[1]}.nc')
            w22_data_loc_dict[k] = os.path.join(sail_winter_22_folder,f'{k}_{winter_22[0]}_{winter_22[1]}.nc')

In [ ]:
w22_SAIL_gts_met = xr.open_dataset(w22_data_loc_dict['met']).sel(time=slice('2021-12-01','2022-03-31'))
w22_SAIL_gts_ld = xr.open_dataset(w22_data_loc_dict['laser_disdrometer_gothic']).sel(time=slice('2021-12-01','2022-03-31'))
w22_SAIL_gts_ldq = xr.open_dataset(w22_data_loc_dict['ldquants']).sel(time=slice('2021-12-01','2022-03-31'))
w22_SAIL_gts_pluvio = xr.open_dataset(w22_data_loc_dict['pluvio2']).sel(time=slice('2021-12-01','2022-03-31'))


### Download winter 2023 data from SAIL
For now we will just get met, ecor, and laser disdrometer data

In [ ]:
# load in the winter 23 data
sail_winter_23_folder = os.path.join(storage_directory,'sail_data',winter_23_folder)
# create empty data dictionary
w23_data_loc_dict = {}
# Iterate through the dictionary and pull the data for each datastream
for k,v in sail_datastream_dict.items():
    if (k =='radiosonde') & (len(os.listdir(os.path.join(sail_winter_23_folder,"radiosonde"))) > 0):
        print("Radiosonde data already donwloaded.")
        print('-------------------')
    # Check if the file already exists
    elif (os.path.exists(f'{sail_winter_23_folder}/{k}_{winter_23[0]}_{winter_23[1]}.nc')): 
        print(f'{k}_{winter_23[0]}_{winter_23[1]}.nc already exists')
        print('-------------------')
        # add the filename to the dictionary which can be used if we want to load the data
        w23_data_loc_dict[k] = os.path.join(sail_winter_23_folder,f'{k}_{winter_23[0]}_{winter_23[1]}.nc')
        continue
    else:
        # explicitly download radiosonde data because they are a lot easier to process and think about when in individual files
        if k == 'radiosonde':
            discovery.download_data(
                api_username,
                api_token,
                v,
                startdate=winter_23[0],
                enddate=winter_23[1],
                output=sail_winter_23_folder+'/radiosonde/'
            )
        else:
            ds = get_sail_data(api_username,
                        api_token,
                        v,
                        startdate=winter_23[0],
                        enddate=winter_23[1])
            ds.to_netcdf(f'{sail_winter_23_folder}/{k}_{winter_23[0]}_{winter_23[1]}.nc')
            w23_data_loc_dict[k] = os.path.join(sail_winter_23_folder,f'{k}_{winter_23[0]}_{winter_23[1]}.nc')

In [ ]:
w23_SAIL_gts_met = xr.open_dataset(w23_data_loc_dict['met']).sel(time=slice('2022-12-01','2023-03-31'))
w23_SAIL_gts_ld = xr.open_dataset(w23_data_loc_dict['laser_disdrometer_gothic']).sel(time=slice('2022-12-01','2023-03-31'))
w23_SAIL_gts_ldq = xr.open_dataset(w23_data_loc_dict['ldquants']).sel(time=slice('2022-12-01','2023-03-31'))
w23_SAIL_gts_pluvio = xr.open_dataset(w23_data_loc_dict['pluvio2']).sel(time=slice('2022-12-01','2023-03-31'))

### Pull in corrected disdrometer data

In [ ]:
w22_SAIL_gts_ld_corrected = xr.open_dataset('../../01_data/processed_data/sail_processed/precipitation_rate_gts_w22.nc')
w23_SAIL_gts_ld_corrected = xr.open_dataset('../../01_data/processed_data/sail_processed/precipitation_rate_gts_w23.nc')

# SPLASH

### Get Tilden's data from the small double fenced intercomparison reference gauge

In [ ]:
# open tilden's precipitaiton data
w23_SPLASH_kp_tilden_prcp = pd.read_csv('../../01_data/processed_data/splash/tilden_precip_2022_2023.csv', index_col=0)


### Get ASFS data

In [ ]:
# open kettle ponds and avery picnic asfs datasets
w22_SPLASH_kp_asfs = xr.open_dataset('../../01_data/processed_data/splash/w22_splash_kp_qc_1H.nc')
w22_SPLASH_ap_asfs = xr.open_dataset('../../01_data/processed_data/splash/w22_splash_ap_qc_1H.nc')
w23_SPLASH_kp_asfs = xr.open_dataset('../../01_data/processed_data/splash/w23_splash_kp_qc_1H.nc')
w23_SPLASH_ap_asfs = xr.open_dataset('../../01_data/processed_data/splash/w23_splash_ap_qc_1H.nc')

### Get Ceilometer Data for Kettle Ponds
- gathered from the [Zenodo site](https://zenodo.org/records/10520198/files/ckp.cl51.cloud_prod.zip?download=1)

In [ ]:
SPLASH_kp_ceil = xr.open_dataset('/storage/dlhogan/synoptic_sublimation/splash_data/ceilometer/splash_kp_ceilometer.nc')
w22_SPLASH_kp_ceil = SPLASH_kp_ceil.sel(time=slice('2021-12-01','2022-03-31'))
w23_SPLASH_kp_ceil = SPLASH_kp_ceil.sel(time=slice('2022-10-01','2023-03-31'))

### Get Corrected Laser Disdrometer Data from KPS
- gathered from the [Zenodo site](https://zenodo.org/records/10372121)

In [ ]:
w22_SPLASH_kp_ld = xr.open_dataset('../../01_data/processed_data/splash/wy22_resampled_1H_SPLASH_kp_ldis.nc')
w23_SPLASH_kp_ld = xr.open_dataset('../../01_data/processed_data/splash/wy23_resampled_1H_SPLASH_kp_ldis.nc')

# filter to the same time period as the ceilometer data
w22_SPLASH_kp_ld = w22_SPLASH_kp_ld.sel(time=slice('2021-12-01','2022-03-31'))
w23_SPLASH_kp_ld = w23_SPLASH_kp_ld.sel(time=slice('2022-12-01','2023-03-31'))

In [ ]:
parsivel_correction_dict = {
    'holroyd1971': [0.17, -1],
    'brandes2007': [0.178, -0.922],
    'heymsfield2004': [0.104, -0.95]
}

def correct_parsivel_for_snow(ds, method='holroyd1971'):
    """
    Correct snowfall rate using a method discussed in Boudala et al. 2014
    """
    a = parsivel_correction_dict[method][0]
    b = parsivel_correction_dict[method][1]
    # Number density of particles
    N_D = ds['particle_distribution']
    # Fall velocity of particles summed over raw_fall_velocity
    # V_D = ds['fall_velocity_calculated']
    # Class size width
    class_size_width = ds['size_bins']

    # Apply the condition to include particle sizes from 2 to 31
    size_bins_indices = range(2, 32)
    raw_fall_velocity_indices = range(2, 32)

    # Select the relevant slices using isel
    N_D_masked = N_D.isel(size_bins=size_bins_indices)
    class_size_width_masked = class_size_width.isel(size_bins=size_bins_indices)
    # V_D_masked = V_D.isel(raw_fall_velocity=raw_fall_velocity_indices)

    # Calculate the snowfall rate using vectorized operations
    result = (N_D_masked * class_size_width_masked ** (3 + b)).sum(dim='size_bins')

    # Calculate the final result
    final_result = (6 * a * np.pi * 10e-3 * result)/60
    return final_result

In [ ]:
# make a new dataset for each with corrected precipitation rates
w22_SPLASH_kp_ld_corrected = correct_parsivel_for_snow(w22_SPLASH_kp_ld)
w23_SPLASH_kp_ld_corrected = correct_parsivel_for_snow(w23_SPLASH_kp_ld)

# billy barr

In [ ]:
# billy barr precipitation data
bb_gts_prcp = pd.read_csv('../../01_data/raw_data/billy_barr/billy_barr_data.xls',skiprows=[1], header=0, delimiter='\t')
# use columns 1 and 2 as the datetime combining date and time
bb_gts_prcp['datetime'] = pd.to_datetime(bb_gts_prcp['Date'] + ' ' + bb_gts_prcp['Time of Day'])
# set the datetime as the index
bb_gts_prcp['datetime'] = pd.to_datetime(bb_gts_prcp['datetime'])

bb_gts_prcp = bb_gts_prcp.set_index('datetime')
# drop the Date and Time of Day columns
bb_gts_prcp = bb_gts_prcp.drop(columns=['Date','Time of Day'])

# winter 22 and 23 data
w22_bb_prcp = bb_gts_prcp.loc['2021-12-01':'2022-03-31']
w23_bb_prcp = bb_gts_prcp.loc['2022-12-01':'2023-03-31']

# Plotting them up

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=w22_bb_prcp.index, 
                         y=w22_bb_prcp['Precipitation'].cumsum(), 
                         name='billy barr', 
                         mode='lines', 
                         line=dict(color='blue')))
fig.add_trace(go.Scatter(x=w22_SPLASH_kp_ld.isel(size_bins=0).time,
                            y=w22_SPLASH_kp_ld['Amount'].isel(size_bins=0).cumsum(),
                            name='SPLASH Kettle Ponds LDIS', 
                            mode='lines', 
                            line=dict(color='red')))
fig.add_trace(go.Scatter(x=w22_SPLASH_kp_ld_corrected.time,
                            y=w22_SPLASH_kp_ld_corrected.cumsum(),
                            name='SPLASH Kettle Ponds LDIS corrected', 
                            mode='lines', 
                            line=dict(color='red', dash='dash'))) 
fig.add_trace(go.Scatter(x=w22_SAIL_gts_pluvio.time,
                            y=w22_SAIL_gts_pluvio['accum_rtnrt'].cumsum(),
                            name='SAIL Gothic Pluvio', 
                            mode='lines', 
                            line=dict(color='green')))
fig.add_trace(go.Scatter(x=w22_SAIL_gts_met.time,
                            y=w22_SAIL_gts_met['tbrg_precip_total'].cumsum(),
                            name='SAIL Gothic Tipping Bucket', 
                            mode='lines', 
                            line=dict(color='purple'))) 
fig.add_trace(go.Scatter(x=w22_SAIL_gts_met.time,
                            y=(w22_SAIL_gts_met['org_precip_rate_mean']/60).cumsum(),
                            name='SAIL Gothic Optical Rain Gauge', 
                            mode='lines', 
                            line=dict(color='orange')))
fig.add_trace(go.Scatter(x=w22_SAIL_gts_met.where(w22_SAIL_gts_met.qc_pwd_precip_rate_mean_1min==0).time,
                            y=(w22_SAIL_gts_met.where(w22_SAIL_gts_met.qc_pwd_precip_rate_mean_1min==0)['pwd_precip_rate_mean_1min']/60).cumsum(),
                            name='SAIL Gothic Present Weather Detector', 
                            mode='lines', 
                            line=dict(color='black')))
fig.add_trace(go.Scatter(x=w22_SAIL_gts_ld.time,
                            y=(w22_SAIL_gts_ld['precip_rate']/60).cumsum(),
                            name='SAIL Gothic Laser Disdrometer (raw)', 
                            mode='lines', 
                            line=dict(color='brown')))
fig.add_trace(go.Scatter(x=w22_SAIL_gts_ld_corrected.time,
                         y=(w22_SAIL_gts_ld_corrected['corrected_prcp_rate_m2']).cumsum(),
                         mode='lines',
                         name='SAIL Gothic Laser Disdrometer (corrected)',
                         line=dict(color='brown', dash='dash')))
fig.add_trace(go.Scatter(x=w22_SAIL_gts_ldq.time,
                            y=(w22_SAIL_gts_ldq['rain_rate']/60).cumsum(),
                            name='SAIL Gothic Laser Disdrometer Quants', 
                            mode='lines', 
                            line=dict(color='gold')))


fig.update_layout(title='Winter 22 Precipitation Comparison',
                    xaxis_title='Date',
                    yaxis_title='Cumulative Precipitation (mm)',
                    showlegend=True,
                    width=800,
                    height=600)
                        

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=w23_bb_prcp[w23_bb_prcp['Precipitation']<10].index, 
                         y=w23_bb_prcp[w23_bb_prcp['Precipitation']<10]['Precipitation'].cumsum(), 
                         name='billy barr', 
                         mode='lines', 
                         line=dict(color='blue')))
fig.add_trace(go.Scatter(x=w23_SPLASH_kp_ld.where(w23_SPLASH_kp_ld['Amount']>0).isel(size_bins=0).time,
                            y=w23_SPLASH_kp_ld.where(w23_SPLASH_kp_ld['Amount']>0).Amount.isel(size_bins=0).cumsum(),
                            name='SPLASH Kettle Ponds LDIS', 
                            mode='lines', 
                            line=dict(color='red')))
fig.add_trace(go.Scatter(x=w23_SPLASH_kp_ld_corrected.time,
                            y=w23_SPLASH_kp_ld_corrected.cumsum(),
                            name='SPLASH Kettle Ponds LDIS corrected', 
                            mode='lines', 
                            line=dict(color='red', dash='dash'))) 
fig.add_trace(go.Scatter(x=w23_SAIL_gts_pluvio.time,
                            y=w23_SAIL_gts_pluvio['accum_rtnrt'].cumsum(),
                            name='SAIL Gothic Pluvio', 
                            mode='lines', 
                            line=dict(color='green')))
fig.add_trace(go.Scatter(x=w23_SAIL_gts_met.time,
                            y=w23_SAIL_gts_met['tbrg_precip_total'].cumsum(),
                            name='SAIL Gothic Tipping Bucket', 
                            mode='lines', 
                            line=dict(color='purple'))) 
fig.add_trace(go.Scatter(x=w23_SAIL_gts_met.time,
                            y=(w23_SAIL_gts_met['org_precip_rate_mean']/60).cumsum(),
                            name='SAIL Gothic Optical Rain Gauge', 
                            mode='lines', 
                            line=dict(color='orange')))
fig.add_trace(go.Scatter(x=w23_SAIL_gts_met.where(w23_SAIL_gts_met.qc_pwd_precip_rate_mean_1min==0).time,
                            y=(w23_SAIL_gts_met.where(w23_SAIL_gts_met.qc_pwd_precip_rate_mean_1min==0)['pwd_precip_rate_mean_1min']/60).cumsum(),
                            name='SAIL Gothic Present Weather Detector', 
                            mode='lines', 
                            line=dict(color='black')))
fig.add_trace(go.Scatter(x=w23_SAIL_gts_ld.time,
                            y=(w23_SAIL_gts_ld['precip_rate']/60).cumsum(),
                            name='SAIL Gothic Laser Disdrometer (raw)', 
                            mode='lines', 
                            line=dict(color='brown')))
fig.add_trace(go.Scatter(x=w23_SAIL_gts_ld_corrected.time,
                         y=(w23_SAIL_gts_ld_corrected['corrected_prcp_rate_m2']).cumsum(),
                         mode='lines',
                         name='SAIL Gothic Laser Disdrometer (corrected)',
                         line=dict(color='brown', dash='dash')))
fig.add_trace(go.Scatter(x=w23_SAIL_gts_ldq.time,
                            y=(w23_SAIL_gts_ldq['rain_rate']/60).cumsum(),
                            name='SAIL Gothic Laser Disdrometer Quants', 
                            mode='lines', 
                            line=dict(color='gold')))
fig.add_trace(go.Scatter(x=w23_SPLASH_kp_tilden_prcp.loc['2022-12-01':'2023-03-31'].index,
                         y=w23_SPLASH_kp_tilden_prcp['Precip_mm'].loc['2022-12-01':'2023-03-31'].cumsum(),
                         name='SPLASH Kettle Ponds SDFIR WB',
                         mode='lines',
                         line=dict(color='magenta')))
                         
fig.update_layout(title='Winter 23 Precipitation Comparison',
                    xaxis_title='Date',
                    yaxis_title='Cumulative Precipitation (mm)',
                    showlegend=True,
                    width=800,
                    height=600)
# add SOS snow pillows to the mix                        

In [ ]:
# do a sum sum plot of the gothic pluvio and billy barr's data